 # **Auto-Correct system**
 
 We will implement an auto-correct system

### Import Libraries

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

### Process Data

- Reading in a corpus (text file)
- Changeing everything to lowercase
- Returning a list of words.

In [2]:
def process_data(file_name):
       
    #Open the file, read its contents into a string variable
    with open(file_name, "r") as f:
        text = f.read()
    
    # convert all letters to lower case
    text = text.lower()
    words = re.findall('\w+',text)
  
    return words

In [3]:
%%capture
! git clone https://github.com/MarwanMohamed95/Auto-Correct

word_l = process_data('/content/Auto-Correct/shakespeare.txt')
vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

### get_count function

- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus.

In [4]:
def get_count(word_l):
    
    word_count_dict = {}  
    for word in word_l:
        if word not in word_count_dict:
            word_count_dict[word] = 1
        else:
            word_count_dict[word] += 1
    return word_count_dict

In [5]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'thee' is 240


### get_probs function
- This returns a dictionary where the keys are words, and the value for each word is its probability in the corpus of words.

In [6]:
def get_probs(word_count_dict):
    
    probs = {}    
    total = 0
    for count in word_count_dict.values():
        total += count
    
    probs = {word:p/total for word,p in word_count_dict.items()}
    return probs

In [7]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


### String Manipulations
we will write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. we will implement four functions:

- delete_letter: given a word, it returns all the possible strings that have one character removed.
- switch_letter: given a word, it returns all the possible strings that have two adjacent letters switched.
- replace_letter: given a word, it returns all the possible strings that have one character replaced by another different letter.
- insert_letter: given a word, it returns all the possible strings that have an additional character inserted.

In [8]:
def delete_letter(word, verbose=False):
    
    split_l = [(word[:i],word[i:]) for i in range(len(word))]
    delete_l = [L+R[1:] for L,R in split_l]

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return  delete_l

delete_word_l = delete_letter(word="cans",verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_l = ['ans', 'cns', 'cas', 'can']


In [9]:
def switch_letter(word, verbose=False):
    
    split_l = [(word[:i],word[i:]) for i in range(len(word))]
    switch_l = [a + b[1] + b[0] + b[2:] for a,b in split_l if len(b) >= 2]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 
    
    return switch_l

switch_word_l = switch_letter(word="eta",verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [10]:
def replace_letter(word, verbose=False):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    
    split_l = [(word[:i],word[i:]) for i in range(len(word))]
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)    
    replace_set.discard(word)
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

replace_l = replace_letter(word='can',verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [11]:
def insert_letter(word, verbose=False):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    insert_l = [a + l + b for a,b in split_l for l in letters]
    
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


### Combining the edits

we will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

In [12]:
def edit_one_letter(word, allow_switches = True):
    
    edit_one_set = set()    
    if allow_switches == False:
        edit_one_set = delete_letter(word) + replace_letter(word) + insert_letter(word) 
    else:
        edit_one_set = delete_letter(word) + replace_letter(word) + insert_letter(word) + switch_letter(word)
    
    return set(edit_one_set)

In [13]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


the `edit_two_letters` function returns a set of words that are two edits away. Creating the additional edits based on the `edit_one_letter` function.

In [14]:
def edit_two_letters(word, allow_switches = True):
    
    edit_two_set = set()    
    edit_one_set = edit_one_letter(word,allow_switches = allow_switches)
    for w in edit_one_set:
        edit_one_set2 = edit_one_letter(w,allow_switches = allow_switches)
        for s in edit_one_set2:
            edit_two_set.add(s) 
    
    return set(edit_two_set)

In [15]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


### get_corrections function 
- This function returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word).

we will generate suggestions for a supplied word using the edit functions we have developed. The 'suggestion algorithm' should follow this logic:

- If the word is in the vocabulary, suggest the word.
- Otherwise, if there are suggestions from edit_one_letter that are in the vocabulary, use those.
- Otherwise, if there are suggestions from edit_two_letters that are in the vocabulary, use those.
- Otherwise, suggest the input word.*

In [16]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    suggestions = []
    n_best = []
    
    word_probs = {}
    
    if word in vocab:
        word_probs[word] = probs[word]
    
    if len(word_probs) == 0:
        one_edit_set = edit_one_letter(word, True)
        for edited_word in one_edit_set:
            if edited_word in vocab:
                word_probs[edited_word] = probs[edited_word]
    
    if len(word_probs) == 0:
        two_edits_set = edit_two_letters(word, True)
        for edited_word in two_edits_set:
            if edited_word in vocab:
                word_probs[edited_word] = probs[edited_word]
    
    n_best = Counter(word_probs).most_common(n)
    
    for best_word, prob in n_best:
        suggestions.append(best_word)
    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [18]:
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

entered word =  dys 
suggestions =  ['days', 'dye']
word 0: days, probability 0.000410
word 1: dye, probability 0.000019
